In [56]:
%pip install scikit-learn


You should consider upgrading via the 'c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [57]:
# from sklearn.feature_selection import SelectPercentile
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
import pandas as pd

##### Train the model from the Bonus Assignment

In [ ]:
df = pd.read_csv("../BonusAssignment/text_training.csv", usecols=list(range(1,2002)))

# separate the features and target variable
features = df.iloc[:, 1:-1] # all columns except the last one (rating)
labels = df.iloc[:, -1] # last column (rating)

selector = SelectPercentile(percentile=10)
x = selector.fit_transform(features, labels)
support = selector.get_feature_names_out()

x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=1)

xgbclf = xgb.XGBClassifier(max_depth=4, seed=2)
xgbclf.fit(x_train, y_train)
y_pred = xgbclf.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


x = features[support]
y_pred = xgbclf.predict(x)
y_pred

In [58]:
df_ffp_train = pd.read_csv("../Documents/ffp_train.csv")
df_reviews_train = pd.read_csv("../Documents/reviews_training.csv")
#df_reviews_train  = pd.read_csv("../Refference/reviews_training.csv")
df_reviews_train

,ID,grey,maker,said,tablespoon,noth,hungri,cancel,shouldnt,anywher,...,medium,truli,fanci,written,person,item,goe,rare,assur,teeth
0,12228,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42793,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42041,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,43815,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,44301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2163,37971,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2164,33722,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2165,3346,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Feature selection

In [59]:
from sklearn.feature_selection import SelectPercentile

In [68]:
features = df_reviews_train.iloc[:, 1:-1] # all columns except the last and the first one
labels = df_reviews_train.iloc[:, -1] # last column (rating)
features
labels

0       0
1       0
2       0
3       0
4       0
       ..
2162    0
2163    0
2164    0
2165    0
2166    0
Name: teeth, Length: 2167, dtype: int64

###### Selection using `SelectPercentile`

In [61]:
selector = SelectPercentile(percentile=19.25)
selector.fit_transform(features, labels)
selected_by_percentile = selector.get_feature_names_out()

print(f"The Number of variables selected: {len(selected_by_percentile)}")
selected_by_percentile

The Number of variables selected: 289


array(['anywher', 'good', 'get', 'could', 'hand', 'idea', 'activ',
       'grade', 'should', 'choke', 'mayb', 'raisin', 'thing', 'prevent',
       'dead', 'for', 'gotten', 'and', 'small', 'plant', 'health',
       'someth', 'these', 'into', 'whenev', 'start', 'other', 'children',
       'from', 'research', 'when', 'suppos', 'updat', 'told', 'thembr',
       'etc', 'arent', 'healthier', 'bit', 'unlik', 'veget', 'soak',
       'process', 'option', 'gag', 'bring', 'choic', 'natur', 'know',
       'babi', 'someon', 'remov', 'age', 'power', 'bear', 'accept',
       'gave', 'hole', 'label', 'nutrit', 'mold', 'except', 'base',
       'myself', 'cancer', 'miner', 'outsid', 'emerg', 'school', 'hous',
       'support', 'member', 'pretzel', 'mention', 'lead', 'stapl', 'life',
       'own', 'today', 'explain', 'anim', 'would', 'are', 'long',
       'difficult', 'homemad', 'than', 'door', 'freez', 'meat', 'fast',
       'crap', 'usual', 'have', 'els', 'often', 'pain', 'special',
       'becaus', 'r

###### Selection using correlation
In Order to Improve our selection process we can eliminate features according to their correlation with each other. This method can be controlled using the threshold.

In [62]:
def select_features(data, threshold):
    correlation = abs(data.corr()["rating"])
    index = correlation > threshold
    # data_new = data[data.columns[index]]
    # data_new["rating"] = data["rating"]
    return data.columns[index]

In [63]:
corr_mat = features.corr()

In [64]:
threshold = 0.4
selected_by_corr = select_features(corr_mat, threshold)

In [65]:
print(f"The Number of variables selected: {len(selected_by_corr)}")
print(f"Selected Threshold: {threshold}")
selected_by_corr

The Number of variables selected: 1202
Selected Threshold: 0.4


['stale',
 'method',
 'tea',
 'soy',
 'canin',
 'occasion',
 'blood',
 'cheaper',
 'mark',
 'breath',
 'expect',
 'duck',
 'skin',
 'sad',
 'higher',
 'digest',
 'horribl',
 'cracker',
 'claim',
 'dollar',
 'opinion',
 'comparison',
 'prepar',
 'plant',
 'mild',
 'activ',
 'batch',
 'plain',
 'import',
 'pretzel',
 'mountain',
 'eater',
 'choke',
 'may',
 'world',
 'itself',
 'recip',
 'develop',
 'inform',
 'blueberri',
 'folk',
 'rich',
 'cane',
 'easili',
 'strip',
 'luck',
 'rins',
 'fine',
 'allerg',
 'huge',
 'watch',
 'sugar',
 'discov',
 'chai',
 'kept',
 'face',
 'mango',
 'ship',
 'immedi',
 'qualiti',
 'instead',
 'third',
 'age',
 'wrong',
 'school',
 'hand',
 'kettl',
 'less',
 'clean',
 'version',
 'includ',
 'wouldnt',
 'option',
 'hair',
 'note',
 'hear',
 'brew',
 'sampl',
 'blue',
 'yummi',
 'piec',
 'agre',
 'fri',
 'made',
 'produc',
 'light',
 'picki',
 'run',
 'today',
 'pull',
 'into',
 'sugarfre',
 'hold',
 'system',
 'taken',
 'juic',
 'cold',
 'sever',
 'orang

In [66]:
set1 = set(selected_by_corr)
set2 = set(selected_by_percentile)

different_values = set1.symmetric_difference(set2)
print("Different values:", different_values)
print("Different values:", len(different_values))

Different values: {'stale', 'method', 'tea', 'seem', 'will', 'soy', 'canin', 'general', 'occasion', 'blood', 'diamond', 'pictur', 'cheaper', 'mark', 'plus', 'breath', 'serious', 'expect', 'addict', 'duck', 'multipl', 'select', 'sad', 'higher', 'horribl', 'thisbr', 'hazelnut', 'cracker', 'flower', 'worst', 'eaten', 'claim', 'part', 'them', 'dollar', 'opinion', 'prepar', 'mild', 'drink', 'batch', 'lack', 'choos', 'plain', 'rais', 'import', 'mountain', 'crack', 'eater', 'program', 'prior', 'excel', 'sweeten', 'raw', 'may', 'world', 'itself', 'recip', 'develop', 'microwav', 'target', 'googl', 'spici', 'pill', 'inform', 'diet', 'blueberri', 'throat', 'folk', 'kick', 'offer', 'boy', 'textur', 'rich', 'sign', 'cane', 'suppli', 'easili', 'strip', 'luck', 'rins', 'cooki', 'fine', 'across', 'allerg', 'written', 'local', 'rid', 'palat', 'huge', 'noth', 'toast', 'balanc', 'sugar', 'discov', 'chai', 'kept', 'hip', 'face', 'mango', 'ship', 'immedi', 'peppermint', 'qualiti', 'although', 'third', 'sup